# JUAS Longitudinal tutorial 1: Getting started with a beam in the center of the RF bucket

Install instructions can be found [on this Github page](github.com/JUAS-tutorials/xsuite-installation).

This notebook will show some longitudinal beam dynamics simulations performed with the code [XSuite](https://github.com/xsuite). Documentation on the code can be found on [https://xsuite.readthedocs.io](https://xsuite.readthedocs.io).

This code is written in [Python](https://www.python.org/). For the tutorials we use  [Jupyter](https://jupyter.org/) notebooks, that can be run in a web browser (with jupyter server) or in an Integrated Development Environment (IDE).

---
Quick navigation hints:

- Notebooks are divided into **cells**. These cells can contains text (like this one), code or special commands (that usually start with !).

- On the **left side**, a color bar appears when clicking on a cell
    - **blue** indicates that the **cell is selected**
    - **green** indicates that the **cell is selected and in edit mode**. The content of the cell can be modified

- Once a cell has been selected, it can be run
    - Using the **run cell** button in the top bar
    - With keyboard keys: <kbd>Ctrl</kbd> + <kbd>Enter</kbd> to run current selected cell
    - With keyboard keys: <kbd>Shift</kbd> + <kbd>Enter</kbd> to run current selected cell and move to the next one

- The cells must **run one after the other**. After a cell has been run, the Python kernel will keep the variables in memory.

In [ ]:
!pip install -r requirements.txt

In [1]:
# Import basic libraries
import numpy as np
from scipy.constants import m_p, c, e

# Import from Xsuite
import xtrack as xt
import xpart as xp
from xpart.longitudinal.rf_bucket import RFBucket

# Matplotlib and animation
import matplotlib.pyplot as plt
import matplotlib.animation as ani

# Matplotlib default parameters for figure size and font size
plt.rcParams.update({'font.size': 16})
plt.rcParams.update({'figure.figsize': (8, 6)})
plt.rcParams.update({'figure.autolayout': True})

# LaTeX printing
from IPython.display import display, Markdown

# import animation functions
from helpers import plot_rf_bucket_animation_persistent, plot_rf_bucket_animation, plot_initial_distribution_and_rf_bucket

# Switch matplotlib backend to qt for interactive plots
%matplotlib tk

# Machine parameters

We must first defined the accelerator parameters: we take the CERN PS parameters as a basis.

In [2]:
####################################
# INPUT HERE Simulation parameters #
####################################

# Machine parameters: circumference and Twiss parameters at injection point
circumference = 100 * 2 * np.pi # in [m]

inj_alpha_x = 0
inj_alpha_y = 0
inj_beta_x = 16.
inj_beta_y = 16.

# Machine parameters: tunes, gamma transition, momentum compaction factor
Qx = 6.3
Qy = 6.4
gamma_tr = 6.1
alpha_c = gamma_tr**(-2)

# Machine parameters: RF voltage, harmonic number, acceleration gradient, bending radius and phase offset
V_rf = 200e3 # in [V]
harmonic = 8

Bdot = 0 # 4.0 # 2.2  # in T/s, try 0.5
bending_radius = 70 # in [m]

# Beam parameters: initial kinetic energy, total intensity, transverse emittances
Ekin = 1.4e9       # in [eV]
intensity = 8e12
epsn_x = 5e-6      # in [m*rad]
epsn_y = 5e-6      # in [m*rad]

# Calculations: Lorentz gamma and beta
gamma = 1 + e * Ekin / (m_p * c**2)
beta = np.sqrt(1 - gamma**-2)

# Beam parameter: initial bunch length
bunch_length = 20e-9  # full bunch length in [s]
sigma_z_0 = (bunch_length / 4) * beta * c  # rms bunch length in [m]

display(Markdown(rf'Initial bunch length: $\sigma_{{z, 0}}={sigma_z_0:.4G}$ m'))

####################################
# END OF INPUT                     #
####################################
# Calculations: total energy
Etot = gamma * m_p * c**2 / e
display(Markdown(rf'$\beta_{{Lorentz}} = {beta:.4f}$   $\gamma_{{Lorentz}} = {gamma:.4f}$ $E_{{total}} = {Etot/1e9:.3G}$ GeV'))

# Calculations: slippage factor and phase offset
eta = alpha_c - gamma**-2
display(Markdown(rf'$\eta = {eta:.4G}$'))

# Calculation: initial momentum and synchrotron rune
p0 = np.sqrt(gamma**2 - 1) * m_p * c
Qs = np.sqrt(np.abs(eta) * V_rf * harmonic/ (2 * np.pi * beta**2 * Etot)) # linear synchrotron tune without acceleration
display(Markdown(rf'$Q_s = {Qs}$'))

# Calculation: revolution time, energy increment per turn
turn_period = circumference / (beta * c)
energy_increment_per_turn = e * circumference * bending_radius * Bdot  # in [J]
energy_increment_per_turn_eV = energy_increment_per_turn / e  # in [eV]

##### RF phase offset calculation #####
# Compute the RF phase offset that is required below and above transition
phi_offset_below = np.arcsin(bending_radius*circumference*Bdot/V_rf)
phi_offset_above = np.pi - phi_offset_below
display(Markdown(rf'RF phase (rad) below transition: $\phi_{{s, below}} = {phi_offset_below:.4G}$ rad'))
display(Markdown(rf'RF phase (rad) above transition: $\phi_{{s, above}} = {phi_offset_above:.4G}$ rad'))

# Convert it to degrees for Xsuite
lag_rf_above = np.rad2deg(phi_offset_above)
lag_rf_below = np.rad2deg(phi_offset_below)
display(Markdown(rf'RF phase below transition: $\phi_{{s, below}} = {lag_rf_below:.4G}^o$'))
display(Markdown(rf'RF phase above transition: $\phi_{{s, above}} = {lag_rf_above:.4G}^o$'))

# Select the RF phase according to the sign of eta
if eta > 0:
    lag_rf = lag_rf_above
else:
    lag_rf = lag_rf_below

Initial bunch length: $\sigma_{z, 0}=1.373$ m

$\beta_{Lorentz} = 0.9160$   $\gamma_{Lorentz} = 2.4921$ $E_{total} = 2.34$ GeV

$\eta = -0.1341$

$Q_s = 0.004172786988495986$

RF phase (rad) below transition: $\phi_{s, below} = 0$ rad

RF phase (rad) above transition: $\phi_{s, above} = 3.142$ rad

RF phase below transition: $\phi_{s, below} = 0^o$

RF phase above transition: $\phi_{s, above} = 180^o$

### Tracking parameters

We define here parameters that are specific to the tracking simulations: the number of particles we want to track, the number of turns in the machine we want to track.

In [3]:
num_particles_to_track = 10
num_turns_to_track = 1000

# The number of frames that will be shown in the animation
# Less frames will make the animation faster
# 50 to 200 is a good range
number_of_frames = 100
save_every = num_turns_to_track // number_of_frames

## Accelerator map

Xsuite provides many elements to model the accelerator. We can have Bending Magnets, Quadrupoles, Sextupoles, RF Cavities etc.
Here we use a LineSegmentMap element: it combines a transverse linear one-turn matrix you saw in the Transverse course, and the longitudinal equation of motion that you saw in the Longitudinal course. We give it the parameters that were defined beforehand.

In [4]:
# Create the one turn transfer matrix
matrix = xt.LineSegmentMap(
            qx=Qx, qy=Qy, # Transverse tunes
            dqx=0, dqy=0, # Transverse chromaticities
            betx=inj_beta_x, alfx=inj_alpha_x, # Horizontal Twiss parameters
            bety=inj_beta_y, alfy=inj_alpha_y, # Vertical Twiss parameters
            dx=0, dpx=0, # Horizontal Dispersion
            dy=0, dpy=0, # Horizontal Dispersion
            voltage_rf=V_rf, # RF voltage
            frequency_rf=harmonic*1/turn_period, # RF frequency
            lag_rf=lag_rf,  # RF phase in degrees for XSuite
            momentum_compaction_factor=gamma_tr**(-2), # Momentum compaction factor
            length=circumference,
            energy_ref_increment=energy_increment_per_turn_eV,
            energy_increment=0,
            )

# Create the line
line = xt.Line(elements=[matrix,])
line.particle_ref = xt.Particles(mass0=xp.PROTON_MASS_EV, gamma0=gamma, q0=1.0)

## Particle distribution

Now that the machine parameters are defined and the one turn map created, we can generate the bunch of particles that will be tracked with Xsuite.

In this example, the initial transverse (x and y) particle distribution is Gaussian.

The longitudinal distribution (z) is also a Gaussian that is matched to the bucket.

In [5]:
# Create the RF bucket object that contains useful information
# such as stable points, bucket height etc.
rf_bucket_properties_dict = {
    "circumference": circumference,
    "gamma": gamma,
    "mass_kg": line.particle_ref.mass0 / (c**2) * e,
    "charge_coulomb": np.abs(line.particle_ref.q0) * e,
    "alpha_array": np.atleast_1d(alpha_c),
    "harmonic_list": np.atleast_1d(harmonic),
    "voltage_list": np.atleast_1d(V_rf),
    "phi_offset_list": np.atleast_1d(lag_rf),
    "p_increment": (energy_increment_per_turn_eV / line.particle_ref.beta0) * (e / c)
}

rfbucket = RFBucket(**rf_bucket_properties_dict)

# Generate the transverse distribution
particle_distribution = xp.generate_matched_gaussian_bunch(line=line, num_particles=num_particles_to_track,
                                                           nemitt_x=epsn_x, nemitt_y=epsn_y, sigma_z=sigma_z_0)

# Modify the particle distribution to have a line distribution instead of a Gaussian
if False:
    particle_distribution.zeta = np.linspace(-5*sigma_z_0, 5*sigma_z_0, num_particles_to_track)
    particle_distribution.delta = np.zeros(num_particles_to_track)

# Save the initial distribution for later
initial_distribution = particle_distribution.copy()

# Call the plot function
# plot_initial_distribution_and_rf_bucket(line, rfbucket, particle_distribution)

## Tracking

We now track the particles through the machine. We will track the particles for `num_turns_to_track` turns.

In [6]:
zeta_value_list = [particle_distribution.zeta.copy()]
delta_value_list = [particle_distribution.delta.copy()]
pzeta_value_list = [particle_distribution.pzeta.copy()]
gamma_value_list = [particle_distribution.gamma0[0].copy()]
p0c_value_list = [particle_distribution.p0c[0].copy()]
kinetic_energy_value_list = [particle_distribution.kinetic_energy0.copy()]
turn_number_list = [0]

for ii_frame in np.arange(number_of_frames+1):

    line.track(particles=particle_distribution,
                num_turns=num_turns_to_track // number_of_frames)

    print(f'Turn {ii_frame*(num_turns_to_track // number_of_frames)}            ', end='\r', flush=True)
    zeta_value_list.append(particle_distribution.zeta.copy())
    delta_value_list.append(particle_distribution.delta.copy())
    pzeta_value_list.append(particle_distribution.pzeta.copy())
    gamma_value_list.append(particle_distribution.gamma0[0].copy())
    p0c_value_list.append(particle_distribution.p0c[0].copy())
    kinetic_energy_value_list.append(particle_distribution.kinetic_energy0.copy())
    turn_number_list.append(ii_frame*(num_turns_to_track // number_of_frames))

## Plotting

We now show the evolution of the particles in the longitudinal phase space.

In [7]:
# Plot an animation of the longitudinal phase space
rf_bucket_properties_dict = {
    "circumference": circumference,
    "gamma": gamma,
    "mass_kg": line.particle_ref.mass0 / (c**2) * e,
    "charge_coulomb": np.abs(line.particle_ref.q0) * e,
    "alpha_array": np.atleast_1d(alpha_c),
    "harmonic_list": np.atleast_1d(harmonic),
    "voltage_list": np.atleast_1d(V_rf),
    "phi_offset_list": np.atleast_1d(lag_rf),
    "p_increment": (energy_increment_per_turn_eV / line.particle_ref.beta0) * (e / c)
}

plot_rf_bucket_animation_persistent(line, gamma_value_list, p0c_value_list, pzeta_value_list, rf_bucket_properties_dict,
zeta_value_list, delta_value_list, turn_number_list)

/home/david/Work/codes/longitudinal-xsuite-tutorial/helpers.py:99: SyntaxWarning: invalid escape sequence '\g'
  ax.set_title(f'Turn {turn_number_list[ii_frame]}\n $\gamma$ {rfbucket.gamma:.1f} $E_{{kin}}$ {(rfbucket.gamma-1)*m_p*c**2/(e*1e9):.2f} GeV')
/home/david/Work/codes/longitudinal-xsuite-tutorial/helpers.py:155: SyntaxWarning: invalid escape sequence '\g'
  ax.set_title(f'Turn {turn_number_list[ii_frame]}\n $\gamma$ {rfbucket.gamma:.1f} $E_{{kin}}$ {(rfbucket.gamma-1)*m_p*c**2/(e*1e9):.2f} GeV')
/home/david/Work/codes/longitudinal-xsuite-tutorial/helpers.py:249: SyntaxWarning: invalid escape sequence '\g'
  plot_title.set_text(f'Turn {turn_number_list[frame_number]}\n $\gamma$ {rfbucket.gamma:.1f} $E_{{kin}}$ {(rfbucket.gamma-1)*m_p*c**2/(e*1e9):.2f} GeV')
/home/david/Work/codes/longitudinal-xsuite-tutorial/helpers.py:357: SyntaxWarning: invalid escape sequence '\g'
  plot_title.set_text(f'Turn {turn_number_list[frame_number]}\n $\gamma$ {rfbucket.gamma:.1f} $E_{{kin}}$ {(rfbuc

TypeError: only 0-dimensional arrays can be converted to Python scalars

In [ ]:
# Plot an animation of the longitudinal phase space
rf_bucket_properties_dict = {
    "circumference": circumference,
    "gamma": gamma,
    "mass_kg": line.particle_ref.mass0 / (c**2) * e,
    "charge_coulomb": np.abs(line.particle_ref.q0) * e,
    "alpha_array": np.atleast_1d(alpha_c),
    "harmonic_list": np.atleast_1d(harmonic),
    "voltage_list": np.atleast_1d(V_rf),
    "phi_offset_list": np.atleast_1d(lag_rf),
    "p_increment": (energy_increment_per_turn_eV / line.particle_ref.beta0) * (e / c)
}

plot_rf_bucket_animation(line, gamma_value_list, p0c_value_list, pzeta_value_list, rf_bucket_properties_dict,
zeta_value_list, delta_value_list, turn_number_list)